#Image augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=20, 
        width_shift_range=[0.5,1.0],
        height_shift_range=[0.5,1.0], 
        zoom_range=[0.8,1.1], 
        brightness_range=[0.7,1.3],
        vertical_flip = False, 
        horizontal_flip=False,
        fill_mode='constant')

In [ ]:
index = ['_0001','_0002']
ext = '.BMP'

for i in range(350):
  label = '%04d'%(i+1)
  fPath = "/content/drive/MyDrive/생체인증보안/face/train2/"+label
  img1 = load_img(fPath+'/'+label+index[0]+ext)
  x = img_to_array(img1)
  x = x.reshape((1,)+x.shape)
  count = 0
  for batch in datagen.flow(x, batch_size=1,save_to_dir=fPath, 
                            save_prefix=fPath+'/'+label+index[0], save_format=ext):
    count+=1
    if count >= 20:
      break;
  img2 = load_img(fPath+'/'+label+index[1]+ext)
  x = img_to_array(img2)
  x = x.reshape((1,)+x.shape)
  count = 0
  for batch in datagen.flow(x, batch_size=1,save_to_dir=fPath,
                            save_prefix=fPath+'/'+label+index[1], save_format=ext):
    count+=1
    if count >= 20:
      break;

#Train 

In [1]:
from google.colab import drive  # 드라이브 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 필요 라이브러리 import 
import numpy as np
import torch
import os
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision
import time
import copy
%matplotlib inline  
%config InlineBackend.figure_format='retina'
print ("PyTorch version:[%s]."%(torch.__version__))

# Device Configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("This notebook use [%s]."%(device))

PyTorch version:[1.8.1+cu101].
This notebook use [cuda:0].


In [4]:
EPOCHS = 100
BATCH_SIZE= 350
LEARNING_RATE=0.001

In [5]:
from torchvision import datasets, transforms, models

transforms = transforms.Compose([transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406],
                                                      [0.229, 0.224, 0.225])])




DATASET_PATH = "/content/drive/MyDrive/생체인증보안/face"


train_data = datasets.ImageFolder(DATASET_PATH + '/train2', transform=transforms)
val_data = datasets.ImageFolder(DATASET_PATH + '/validate', transform=transforms)

train_iter = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
val_iter = torch.utils.data.DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)

In [6]:
pretrained_model = models.resnet18(pretrained=True)
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 350) #분류할 class 350개 

pretrained_model = pretrained_model.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_model.parameters(), lr=LEARNING_RATE)

In [8]:
total_params = 0
for param_name, param in pretrained_model.named_parameters():
    if param.requires_grad:
        total_params += len(param.reshape(-1))
print(f"Number of Total Parameters: {total_params:,d}")

Number of Total Parameters: 11,356,062


In [9]:
def test_eval(model, data_iter, batch_size):
    with torch.no_grad():
        test_loss = 0
        total = 0
        correct = 0
        model.eval()
        for batch_img, batch_lab in data_iter:
            X = batch_img.to(device)
            Y = batch_lab.to(device)
            y_pred = model(X)
            _, predicted = torch.max(y_pred.data, 1)
            correct += (predicted == Y).sum().item()
            total += batch_img.size(0)
        val_acc = (100 * correct / total)
        model.train()
    return val_acc

In [11]:
# Training Phase
print_every = 1
print("Start training !")

# Training loop
for epoch in range(EPOCHS):
    loss_val_sum = 0
    for batch_img, batch_lab in train_iter:

        X = batch_img.to(device)
        Y = batch_lab.to(device)
        
        y_pred = pretrained_model.forward(X)
        loss = criterion(y_pred, Y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_val_sum += loss
        
    if ((epoch%print_every)==0) or (epoch==(EPOCHS-1)):
        loss_val_avg = loss_val_sum / len(train_iter)
        accr_val = test_eval(pretrained_model, val_iter, BATCH_SIZE)
        print(f"epoch:[{epoch+1}/{EPOCHS}] cost:[{loss_val_avg:.3f}] test_accuracy:[{accr_val:.3f}]")

print("Training Done !")

Start training !
epoch:[1/100] cost:[0.109] test_accuracy:[92.286]
epoch:[2/100] cost:[0.005] test_accuracy:[93.714]
epoch:[3/100] cost:[0.002] test_accuracy:[94.000]
epoch:[4/100] cost:[0.001] test_accuracy:[94.571]
epoch:[5/100] cost:[0.001] test_accuracy:[94.286]
epoch:[6/100] cost:[0.001] test_accuracy:[94.571]
epoch:[7/100] cost:[0.001] test_accuracy:[94.571]
epoch:[8/100] cost:[0.001] test_accuracy:[94.857]
epoch:[9/100] cost:[0.001] test_accuracy:[94.857]
epoch:[10/100] cost:[0.001] test_accuracy:[94.857]
epoch:[11/100] cost:[0.000] test_accuracy:[94.857]
epoch:[12/100] cost:[0.000] test_accuracy:[95.143]
epoch:[13/100] cost:[0.000] test_accuracy:[94.857]
epoch:[14/100] cost:[0.000] test_accuracy:[94.857]
epoch:[15/100] cost:[0.000] test_accuracy:[94.857]
epoch:[16/100] cost:[0.000] test_accuracy:[95.143]
epoch:[17/100] cost:[0.000] test_accuracy:[95.143]
epoch:[18/100] cost:[0.000] test_accuracy:[95.143]
epoch:[19/100] cost:[0.000] test_accuracy:[95.143]
epoch:[20/100] cost:[0.

In [12]:
val_iter = torch.utils.data.DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
data_iter = iter(val_iter)
images, labels = next(data_iter)

In [13]:
def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')
    ax.set_title(title)
    return ax
  

In [14]:
inputs, classes = next(iter(train_iter))
out = torchvision.utils.make_grid(inputs)

In [15]:
n_sample = 350
test_x = images[:n_sample]
test_y = labels[:n_sample]

with torch.no_grad():
    pretrained_model.eval()
    y_pred = pretrained_model.forward(test_x.type(torch.float).to(device))
    pretrained_model.train()
    
y_pred = y_pred.argmax(axis=1)

test_accuracy = sum([1 for i, j in zip(y_pred, test_y) if i == j]) / n_sample
print(f"test_accuracy : {test_accuracy:.3f}")

for idx in range(n_sample):
    print("Predict:",str(y_pred[idx]+1)[7:10], "Label:",str(test_y[idx]+1)[7:10].replace(')', ''))

test_accuracy : 0.951
Predict: 108 Label: 108
Predict: 325 Label: 325
Predict: 137 Label: 137
Predict: 301 Label: 301
Predict: 136 Label: 136
Predict: 288 Label: 288
Predict: 254 Label: 254
Predict: 226 Label: 226
Predict: 51, Label: 51
Predict: 62, Label: 62
Predict: 55, Label: 55
Predict: 210 Label: 210
Predict: 94, Label: 94
Predict: 193 Label: 193
Predict: 83, Label: 83
Predict: 143 Label: 143
Predict: 306 Label: 306
Predict: 172 Label: 172
Predict: 48, Label: 48
Predict: 31, Label: 31
Predict: 101 Label: 34
Predict: 170 Label: 170
Predict: 77, Label: 77
Predict: 331 Label: 331
Predict: 168 Label: 168
Predict: 203 Label: 203
Predict: 310 Label: 310
Predict: 211 Label: 211
Predict: 239 Label: 239
Predict: 52, Label: 52
Predict: 251 Label: 251
Predict: 169 Label: 169
Predict: 343 Label: 343
Predict: 44, Label: 44
Predict: 163 Label: 163
Predict: 74, Label: 334
Predict: 202 Label: 208
Predict: 123 Label: 123
Predict: 213 Label: 213
Predict: 243 Label: 243
Predict: 230 Label: 230
Predi

#Test

##Test dataset1 (1~350) 실행




In [16]:
test_data = datasets.ImageFolder(DATASET_PATH + '/test1', transform=transforms)
test_iter = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

class_name = test_data.classes
class_len = len(class_name)

In [17]:
data_iter = iter(test_iter)
images, labels = next(data_iter)

# 라벨링할 class는 train_iter에서
inputs, classes = next(iter(train_iter))
out = torchvision.utils.make_grid(inputs)

n_sample = 350
test_x = images[:n_sample]
test_y = labels[:n_sample]

with torch.no_grad():
    pretrained_model.eval()
    y_pred = pretrained_model.forward(test_x.type(torch.float).to(device))
    pretrained_model.train()
    
y_pred = y_pred.argmax(axis=1)

result = []

for idx in range(n_sample):
    print( "File name:",str(class_name[test_y[idx]]),"Predict:",str(y_pred[idx]+1)[7:10])
    result.append({'image':class_name[test_y[idx]], 'answer':y_pred[idx]+1})



File name: 1 Predict: 5, 
File name: 10 Predict: 153
File name: 100 Predict: 231
File name: 101 Predict: 1, 
File name: 102 Predict: 132
File name: 103 Predict: 51,
File name: 104 Predict: 326
File name: 105 Predict: 107
File name: 106 Predict: 51,
File name: 107 Predict: 87,
File name: 108 Predict: 228
File name: 109 Predict: 224
File name: 11 Predict: 102
File name: 110 Predict: 29,
File name: 111 Predict: 4, 
File name: 112 Predict: 227
File name: 113 Predict: 251
File name: 114 Predict: 294
File name: 115 Predict: 324
File name: 116 Predict: 31,
File name: 117 Predict: 119
File name: 118 Predict: 194
File name: 119 Predict: 280
File name: 12 Predict: 266
File name: 120 Predict: 332
File name: 121 Predict: 241
File name: 122 Predict: 188
File name: 123 Predict: 319
File name: 124 Predict: 215
File name: 125 Predict: 271
File name: 126 Predict: 114
File name: 127 Predict: 239
File name: 128 Predict: 57,
File name: 129 Predict: 158
File name: 13 Predict: 216
File name: 130 Predict: 28

In [18]:
from openpyxl import Workbook # 값을 자동으로 엑셀에 저장하는 코드
 
wb = Workbook()

sheet1 = wb.active

sheet1.title = 'sampleSheet'
fname = 'result_face_test2_data_100.xlsx'

n = len(result)

for i in range(n):
  sheet1.cell(row=i+1, column=1).value = result[i].get('image')
  sheet1.cell(row=i+1, column=2).value =  str(result[i].get('answer'))[7:10].replace(',','')

wb.save(filename=fname)


##Test dataset2 (351~700) 실행

In [19]:
test_data = datasets.ImageFolder(DATASET_PATH + '/test2', transform=transforms)
test_iter = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

class_name = test_data.classes
class_len = len(class_name)

In [20]:
data_iter = iter(test_iter)
images, labels = next(data_iter)

# 라벨링할 class는 train_iter에서
inputs, classes = next(iter(train_iter))
out = torchvision.utils.make_grid(inputs)

n_sample = 350
test_x = images[:n_sample]
test_y = labels[:n_sample]

with torch.no_grad():
    pretrained_model.eval()
    y_pred = pretrained_model.forward(test_x.type(torch.float).to(device))
    pretrained_model.train()
    
y_pred = y_pred.argmax(axis=1)

result2 = []


for idx in range(n_sample):
    print("File name:",str(class_name[test_y[idx]]),"Predict:",str(y_pred[idx]+1)[7:10])
    result2.append({'image':class_name[test_y[idx]], 'answer':y_pred[idx]+1})  

File name: 351 Predict: 329
File name: 352 Predict: 232
File name: 353 Predict: 26,
File name: 354 Predict: 105
File name: 355 Predict: 6, 
File name: 356 Predict: 130
File name: 357 Predict: 301
File name: 358 Predict: 232
File name: 359 Predict: 231
File name: 360 Predict: 214
File name: 361 Predict: 349
File name: 362 Predict: 24,
File name: 363 Predict: 100
File name: 364 Predict: 187
File name: 365 Predict: 88,
File name: 366 Predict: 229
File name: 367 Predict: 41,
File name: 368 Predict: 234
File name: 369 Predict: 87,
File name: 370 Predict: 195
File name: 371 Predict: 80,
File name: 372 Predict: 61,
File name: 373 Predict: 65,
File name: 374 Predict: 13,
File name: 375 Predict: 294
File name: 376 Predict: 312
File name: 377 Predict: 86,
File name: 378 Predict: 148
File name: 379 Predict: 85,
File name: 380 Predict: 314
File name: 381 Predict: 214
File name: 382 Predict: 75,
File name: 383 Predict: 232
File name: 384 Predict: 96,
File name: 385 Predict: 33,
File name: 386 Predi

In [21]:
wb = Workbook()

sheet1 = wb.active

sheet1.title = 'sampleSheet'
fname = 'result_face_test2_data2_100.xlsx'

n = len(result2)

for i in range(n):
  sheet1.cell(row=i+1, column=1).value = result2[i].get('image')
  r = str(result2[i].get('answer'))[7:10].replace(',','')
  r = r.replace(')','')
  sheet1.cell(row=i+1, column=2).value =  r

wb.save(filename=fname)